In [ ]:
import pandas as pd
import numpy as np
import sys

sys.path.append('..')

from interpretDistill.fourierDistill import *
from interpretDistill.binaryTransformer import *

In [ ]:
def process_csv(df):
    y_true = df['y_true']
    y_hat = df['y_hat']
    X = df.drop(columns = ['id', 'y_true', 'y_hat'])
    return X, y_true, y_hat

In [ ]:
Xy_concept_train = pd.read_csv('data/Xy_concept_train.csv', index_col = [0])
X_concept_train, y_concept_train_true, y_concept_train_hat = process_csv(Xy_concept_train)

In [ ]:
ftd = FTDistillClassifierCV(size_interactions=2)

In [ ]:
ftd.fit(X_concept_train, y_concept_train_hat)

In [ ]:
ftd.scores_

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 2, interaction_only = True)


In [ ]:
poly.fit(X_concept_train)

In [ ]:
poly_features = list(map(lambda s: set(s.split()), poly.get_feature_names_out(X_concept_train.columns)))

In [ ]:
poly.transform(X_concept_train).shape

In [ ]:
Chi = pd.DataFrame(poly.transform(X_concept_train), columns = list(map(lambda f: tuple(f), poly_features)))

In [ ]:
y = y_concept_train_hat

In [ ]:
scores_ = [[] for _ in [0.01, 0.1, 1]]
#scorer = kwargs.get("scoring", log_loss)
kf = KFold(n_splits=2)
for i, (train_index, test_index) in enumerate(kf.split(Chi)):
    print(i)
    Chi_out, y_out = Chi.iloc[test_index, :], y.iloc[test_index]
    Chi_in, y_in = Chi.iloc[train_index, :], y.iloc[train_index]
    for i, reg_param in enumerate([0.01, 0.1, 1]):
        base_est = LogisticRegression(C = 1/reg_param, penalty = 'l1', max_epochs = 50000, max_iter=50)
        base_est.fit(Chi_in, y_in)
        scores_[i].append(np.mean(base_est.predict(Chi_out) == y_out))
scores_ = [np.mean(s) for s in scores_]

reg_param = reg_param_list[np.argmax(scores_)]
regression_model = LogisticRegression(C = 1/reg_param, penalty = 'l1', max_epochs = 5000, max_iter=100)
regression_model.fit(Chi, y)